In [1]:
import tensorflow as tf
from typing import List, Tuple

2025-04-09 15:17:42.247289: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-09 15:17:42.368931: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-09 15:17:49.702906: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/iscb/wolfson/doririmon/anaconda3/

In [2]:
import sys
sys.path.append('../../../../')

In [30]:
tf.config.set_visible_devices([], 'GPU')

In [107]:
# Example input
inputs = tf.keras.Input(shape=(11,))
x = tf.keras.layers.Dense(1024, activation='relu')(inputs)
x = TransformerEncoderMLP(hidden_units=(1024, 1024), dropout_rate=0.2)(x)
model = tf.keras.Model(inputs, x)
model.summary()

Model: "model_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 11)]              0         
                                                                 
 dense_185 (Dense)           (None, 1024)              12288     
                                                                 
 transformer_encoder_mlp_23   (None, 1024)             2101248   
 (TransformerEncoderMLP)                                         
                                                                 
Total params: 2,113,536
Trainable params: 2,113,536
Non-trainable params: 0
_________________________________________________________________


In [108]:
model(tf.random.uniform((10, 11)))

<tf.Tensor: shape=(10, 1024), dtype=float32, numpy=
array([[ 0.90425795,  0.21220875,  0.12675744, ..., -1.2134382 ,
        -0.41431978, -1.2155956 ],
       [ 2.0990896 ,  0.42351112,  0.23918264, ..., -1.4000756 ,
         0.7024706 , -1.4182214 ],
       [ 0.19723973, -0.4140548 ,  0.6895801 , ..., -0.9461419 ,
         0.39200586, -1.8634183 ],
       ...,
       [-0.07631502, -0.48998123, -0.30358535, ..., -1.0339234 ,
        -0.07782497, -0.83783656],
       [-0.14333615, -1.0692644 , -0.4346997 , ..., -1.1219832 ,
        -0.1697183 , -0.93750757],
       [-0.12023279, -0.729512  ,  0.15584618, ..., -0.992909  ,
         0.914152  , -0.83675957]], dtype=float32)>

In [109]:
from models.patch_to_score.models.pts_encoder_mlps.utils import GlobalSumPooling

In [110]:
from typing import Tuple
# import tensorflow as tf
# from models.patch_to_score.models.pts_without_mlp_b.utils import GlobalSumPooling


def broadcast_shape(x, max_number_of_patches: int) -> tf.Tensor:
    x_expanded = tf.expand_dims(x, axis=1)
    x_broadcasted = tf.broadcast_to(x_expanded, [tf.shape(x)[0], max_number_of_patches, tf.shape(x)[-1]])
    return x_broadcasted


def build_model_prev(m_a: int, m_c: int, n_layers: int, input_shape: Tuple[int, int],
                max_number_of_patches: int) -> tf.keras.models.Model:
    '''
    :param m_a: size of the hidden layers in the MLP of the components
    :param m_c: size of the hidden layers in the MLP of the concatenated global sum output and size + n_patches MLP output
    :param n_layers: number of layers in each of the MLPs
    :param input_shape: shape of the input data (number of patches, number of features)
    :param max_number_of_patches: maximum number of patches
    :return: a Keras model
    '''
    input_data = tf.keras.Input(shape=input_shape, name='patches_input')
    size_value = tf.keras.Input(shape=(1,), name='extra_value_input')
    n_patches_hot_encoded_value = tf.keras.Input(
        shape=(max_number_of_patches + 1,), name='hot_encoded_value_input')
    n_patches = tf.argmax(n_patches_hot_encoded_value, axis=1)[..., None]
    n_patches = tf.cast(n_patches, tf.float32)
    
    n_patches_broadcased = broadcast_shape(n_patches, max_number_of_patches)
    size_broadcased = broadcast_shape(size_value, max_number_of_patches)
    
    concat_input_data = tf.keras.layers.Concatenate(axis=-1)([input_data, n_patches_broadcased, size_broadcased])
    masked_input = tf.keras.layers.Masking(mask_value=0.0)(concat_input_data)

    currentOutput = masked_input
    for i in range(n_layers):
        dense_output = tf.keras.layers.Dense(
            m_a, activation='linear')(currentOutput)
        batchNorm = tf.keras.layers.BatchNormalization(
            momentum=0.75)(dense_output)
        activation = tf.keras.layers.ReLU()(batchNorm)
        currentOutput = activation


    print('before global pooling', currentOutput)
    global_pooling_output = GlobalSumPooling(
        data_format='channels_last')(currentOutput)
    print('after global pooling shape:', global_pooling_output)

    currentOutput = global_pooling_output
    for i in range(n_layers):
        dense_output = tf.keras.layers.Dense(
            m_c, activation='linear')(currentOutput)
        batchNorm = tf.keras.layers.BatchNormalization(
            momentum=0.75)(dense_output)
        activation = tf.keras.layers.ReLU()(batchNorm)
        currentOutput = activation

    before_sigmoid_output = currentOutput

    output = tf.keras.layers.Dense(
        1, activation='sigmoid')(before_sigmoid_output)
    model = tf.keras.Model(
        inputs=[input_data, size_value, n_patches_hot_encoded_value], outputs=output)
    return model


In [ ]:
from typing import Tuple, List
import tensorflow as tf
from models.patch_to_score.models.pts_encoder_mlps.transformer_encoder_mlp import TransformerEncoderMLP


def broadcast_shape(x, max_number_of_patches: int) -> tf.Tensor:
    x_expanded = tf.expand_dims(x, axis=1)
    x_broadcasted = tf.broadcast_to(x_expanded, [tf.shape(x)[0], max_number_of_patches, tf.shape(x)[-1]])
    return x_broadcasted


def build_model(hidden_sizes_mlp_a: List[Tuple[int, int]], mlp_a_dropout_rate: float,
                hidden_sizes_mlp_c: List[Tuple[int, int]], mlp_c_dropout_rate: float,
                activation: str,
                input_shape: Tuple[int, int],
                max_number_of_patches: int) -> tf.keras.models.Model:
    '''
    :param m_a: size of the hidden layers in the MLP of the components
    :param m_c: size of the hidden layers in the MLP of the concatenated global sum output and size + n_patches MLP output
    :param n_layers: number of layers in each of the MLPs
    :param input_shape: shape of the input data (number of patches, number of features)
    :param max_number_of_patches: maximum number of patches
    :return: a Keras model
    '''
    input_data = tf.keras.Input(shape=input_shape, name='patches_input')
    size_value = tf.keras.Input(shape=(1,), name='extra_value_input')
    n_patches_hot_encoded_value = tf.keras.Input(
        shape=(max_number_of_patches + 1,), name='hot_encoded_value_input')
    n_patches = tf.argmax(n_patches_hot_encoded_value, axis=1)[..., None]
    n_patches = tf.cast(n_patches, tf.float32)
    
    n_patches_broadcased = broadcast_shape(n_patches, max_number_of_patches)
    size_broadcased = broadcast_shape(size_value, max_number_of_patches)
    
    concat_input_data = tf.keras.layers.Concatenate(axis=-1)([input_data, n_patches_broadcased, size_broadcased])
    masked_input = tf.keras.layers.Masking(mask_value=0.0)(concat_input_data)
    
    current_output = masked_input
    print(current_output.shape)
    current_output = tf.keras.layers.Dense(
        hidden_sizes_mlp_a[0][0], activation='linear')(current_output)
    
    for mlp_hidden_size in hidden_sizes_mlp_a:
        mlp = TransformerEncoderMLP(
            hidden_units=mlp_hidden_size, dropout_rate=mlp_a_dropout_rate, activation=activation)
        current_output = mlp(current_output)
        
        # current_output = apply_transformer_mlp(current_output, mlp_hidden_size, mlp_a_dropout_rate, activation)
        # activation = tf.keras.layers.ReLU()(current_output)
        # current_output = activation
        # TODO: apply non-linearity here?
    
    print('before global pooling', current_output)
    
    global_pooling_output = GlobalSumPooling(
        data_format='channels_last')(current_output)

    print('after global pooling:', global_pooling_output)
    
    current_output = global_pooling_output
    current_output = tf.keras.layers.Dense(
        hidden_sizes_mlp_c[0][0], activation='linear')(current_output)
    
    print('here')
    for mlp_hidden_size in hidden_sizes_mlp_c:
        mlp = TransformerEncoderMLP(
            hidden_units=mlp_hidden_size, dropout_rate=mlp_c_dropout_rate, activation=activation)
        current_output = mlp(current_output)
        
        # current_output = apply_transformer_mlp(current_output, mlp_hidden_size, mlp_a_dropout_rate, activation)
        # TODO: apply non-linearity here?

    before_sigmoid_output = current_output

    output = tf.keras.layers.Dense(
        1, activation='sigmoid')(before_sigmoid_output)
    model = tf.keras.Model(
        inputs=[input_data, size_value, n_patches_hot_encoded_value], outputs=output)
    return model


In [112]:
def bootstrap_pts_encoder_mlps(hidden_sizes_mlp_a: List[Tuple[int, int]], mlp_a_dropout_rate: float,
                               hidden_sizes_mlp_c: List[Tuple[int, int]], mlp_c_dropout_rate: float,
                               activation: str, input_shape: Tuple[int, int],
                               max_number_of_patches: int) -> tf.keras.models.Model:
    model = build_model(hidden_sizes_mlp_a, mlp_a_dropout_rate,
                                         hidden_sizes_mlp_c, mlp_c_dropout_rate,
                                         activation, input_shape, max_number_of_patches)
    return model

In [113]:
build_model_prev(1024, 1024, 4, [10, 9], 10)

before global pooling KerasTensor(type_spec=TensorSpec(shape=(None, 10, 1024), dtype=tf.float32, name=None), name='re_lu_94/Relu:0', description="created by layer 're_lu_94'")
mask is not None
after global pooling shape: KerasTensor(type_spec=TensorSpec(shape=(None, 1024), dtype=tf.float32, name=None), name='global_sum_pooling_17/Sum:0', description="created by layer 'global_sum_pooling_17'")


In [114]:
build_model(
    hidden_sizes_mlp_a=[(1024, 1024), (1024, 1024)],
    mlp_a_dropout_rate=0.2,
    hidden_sizes_mlp_c=[(1024, 1024), (1024, 1024)],
    mlp_c_dropout_rate=0.2,
    activation='relu',
    input_shape=[10, 9],
    max_number_of_patches=10)

(None, 10, 11)
before global pooling KerasTensor(type_spec=TensorSpec(shape=(None, 10, 1024), dtype=tf.float32, name=None), name='transformer_encoder_mlp_25/layer_normalization_31/add:0', description="created by layer 'transformer_encoder_mlp_25'")
mask is not None
after global pooling: KerasTensor(type_spec=TensorSpec(shape=(None, 1024), dtype=tf.float32, name=None), name='global_sum_pooling_18/Sum:0', description="created by layer 'global_sum_pooling_18'")
here


In [104]:
# bootstrap_pts_encoder_mlps(
#     hidden_sizes_mlp_a=[(1024, 1024), (1024, 1024)],
#     mlp_a_dropout_rate=0.2,
#     hidden_sizes_mlp_c=[(1024, 1024), (1024, 1024)],
#     mlp_c_dropout_rate=0.2,
#     activation='relu',
#     input_shape=[10, 9],
#     max_number_of_patches=10
# )